In [1]:

%pwd

'c:\\Users\\numai\\Documents\\endtoend\\cnn_classifier\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\numai\\Documents\\endtoend\\cnn_classifier'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_files: Path
    unzip_dir: Path

In [5]:
import cnn_classifier.utils.common as common
print(common.__file__)

import cnn_classifier
print(cnn_classifier.__file__)

C:\Users\numai\Documents\endtoend\cnn_classifier\src\cnn_classifier\utils\common.py
C:\Users\numai\Documents\endtoend\cnn_classifier\src\cnn_classifier\__init__.py


In [6]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_files=config.local_data_files,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
    

In [7]:
import os
import urllib.request as request
import zipfile
from cnn_classifier.utils.common import get_size
from cnn_classifier import logger



class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    def download_files(self):
        if not os.path.exists(self.config.local_data_files):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_files
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_files))}")
        
    def extract_unzip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_files,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_files()
    data_ingestion.extract_unzip_file()
except Exception as e:
    raise e
    

[2026-02-23 02:51:39,691: INFO: common: READ_YAML_VERSION=v3 | reading: C:\Users\numai\Documents\endtoend\cnn_classifier\config\config.yaml]
[2026-02-23 02:51:39,693: INFO: common: yaml file loaded successfully: C:\Users\numai\Documents\endtoend\cnn_classifier\config\config.yaml]
[2026-02-23 02:51:39,694: INFO: common: READ_YAML_VERSION=v3 | reading: C:\Users\numai\Documents\endtoend\cnn_classifier\params.yaml]
[2026-02-23 02:51:39,696: INFO: common: yaml file loaded successfully: C:\Users\numai\Documents\endtoend\cnn_classifier\params.yaml]
[2026-02-23 02:51:39,697: INFO: common: created directory at: artifacts]
[2026-02-23 02:51:39,698: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-23 02:51:48,474: INFO: 2247119136: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
